In [ ]:
import os
import json
import logging
from pathlib import Path
from typing import List

import pandas as pd
from openai import OpenAI

from prompts import PROMPT_GUIA_RESOLUCAO, REGRAS_SAIDA
from src.codes.questao import Questao

import time


In [40]:
# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)


In [41]:
# Inicialização do cliente OpenAI
client = OpenAI(api_key=os.getenv('API_KEY'))

# Caminhos
CAMINHO_DADOS = Path("src/data/base_questoes_classificadas_filtrada.csv")
CAMINHO_SAIDA = Path("src/data/respostas_questoes.csv")

In [42]:
def carregar_base(caminho: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(caminho, sep=',', encoding='utf-8')
    except Exception as e:
        logging.error(f"Erro ao carregar a base: {e}")
        raise

In [43]:
def gerar_prompt(enunciado: str, assunto: str, grupo_subassunto: str, subassunto: str, nivel_dificuldade: str) -> str:
    return PROMPT_GUIA_RESOLUCAO.format(
        enunciado=enunciado,
        assunto=assunto,
        grupo_subassunto=grupo_subassunto,
        subassunto=subassunto,
        nivel_dificuldade=nivel_dificuldade,
        regras_da_saida=REGRAS_SAIDA.format(regra="Uma explicação detalhada da resolução da questão.")
    )

In [44]:
def obter_resposta(prompt: str, model: str = 'gpt-4o-mini-2024-07-18') -> str:
    try:
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logging.error(f"Erro ao gerar resposta do modelo: {e}")
        return "Erro ao obter resposta."

In [48]:
def processar_questoes(df: pd.DataFrame) -> List[dict]:
	respostas = []

	for i in range(len(df)):
		try:
			row = df.iloc[i]
			enunciado = Questao(row['url']).texto
			prompt = gerar_prompt(
				enunciado=enunciado,
				assunto=row['assunto'],
				grupo_subassunto=row['grupo_subassunto'],
				subassunto=row['subassunto'],
				nivel_dificuldade=row['nivel']
			)
			resposta = obter_resposta(prompt)

			logging.info(f"------------ Questão {i + 1}/{len(df)} processada com sucesso. ------------")
			print(f'Assunto: {row["assunto"]}, Subassunto: {row["subassunto"]}, Nível: {row["nivel"]}')
			print(f'Resposta: {resposta}')

			respostas.append({
				'url': row['url'],
				'assunto': row['assunto'],
				'grupo_subassunto': row['grupo_subassunto'],
				'subassunto': row['subassunto'],
				'nivel': row['nivel'],
				'nivel_subassunto': row['nivel_subassunto'],
				'resposta': resposta
			})
			time.sleep(10)  # Atraso para evitar sobrecarga na API
		except Exception as e:
			logging.error(f"Erro ao processar questão {i + 1}: {e}")

	return respostas


In [46]:
def salvar_respostas(respostas: List[dict], caminho_saida: Path) -> None:
    df_respostas = pd.DataFrame(respostas)
    try:
        df_respostas.to_csv(caminho_saida, sep=',', encoding='utf-8', index=False)
        logging.info(f"Arquivo salvo com sucesso em: {caminho_saida}")
    except Exception as e:
        logging.error(f"Erro ao salvar arquivo: {e}")

In [49]:
base_questoes = carregar_base(CAMINHO_DADOS)
respostas = processar_questoes(base_questoes)
salvar_respostas(respostas, CAMINHO_SAIDA)

2025-06-07 15:46:27,392 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-07 15:46:27,396 - INFO - ------------ Questão 1/139 processada com sucesso. ------------
2025-06-07 15:46:27,398 - ERROR - Erro ao processar questão 1: name 'time' is not defined


Assunto: Algoritmos e Estruturas de Dados, Subassunto: Estratégias simples de iteração e repetição, Nível: Médio
Resposta: Para resolver a questão apresentada, siga as etapas abaixo:

1. **Leitura do Enunciado**: Antes de tudo, é essencial que você compreenda bem o que está sendo pedido. Identifique quais são os dados de entrada e qual é a saída esperada. Tente destacar palavras-chave ou frases que indiquem operações específicas que você precisa realizar.

2. **Identificação dos Desafios**: Pense nos aspectos que podem ser desafiadores na questão, como manipulação de dados, iterações, condições e formatos de saída. Estabeleça quais são os pontos que você não está muito certo de como abordar.

3. **Estratégia de Solução**: Comece a pensar em como você pode estruturar sua solução. Uma boa analogia para iteração é imaginar que você está contando dinheiro: você precisa verificar cada nota ou moeda (que pode ser uma parte da lista ou conjunto de dados) para saber o valor total. Considere qu

2025-06-07 15:46:38,961 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-07 15:46:38,964 - INFO - ------------ Questão 2/139 processada com sucesso. ------------
2025-06-07 15:46:38,965 - ERROR - Erro ao processar questão 2: name 'time' is not defined


Assunto: Fundamentos de Matemática, Subassunto: Números inteiros, operações (incluindo exponenciação) e comparações, Nível: Médio
Resposta: Para resolver a questão, siga as etapas a seguir:

1. **Leitura Atenta do Enunciado**: Comece lendo o enunciado cuidadosamente para que você compreenda o que está sendo solicitado. Identifique os números ou variáveis envolvidas, as operações a serem realizadas e as condições mencionadas.

2. **Identificação dos Desafios**: Note que a questão envolve números inteiros, operações matemáticas e comparações. Pense sobre quais operações devem ser realizadas e se há alguma regra específica para essas operações.

3. **Dicas Contextualizadas**:
   - Imagine uma situação em que você precisa comparar alturas de pessoas. Assim, ao invés de simplesmente fazer cálculos, pense sobre a razão pela qual você está fazendo essas operações e comparações — isso pode ajudar a visualizar o problema.

4. **Passo a Passo da Lógica**:
   - Primeiro, defina quais são as entra

2025-06-07 15:46:51,478 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-07 15:46:51,483 - INFO - ------------ Questão 3/139 processada com sucesso. ------------
2025-06-07 15:46:51,485 - ERROR - Erro ao processar questão 3: name 'time' is not defined


...

Assunto: Algoritmos e Estruturas de Dados, Subassunto: Pilha e Fila, Nível: Médio
Resposta: Para abordar essa questão que envolve Pilhas e Filas, siga os passos abaixo:

1. **Leitura Atenta do Enunciado**: Identifique quais operações devem ser realizadas sobre as estruturas (por exemplo, push, pop para pilhas, enfileirar, desenfileirar para filas) e quais condições devem ser atendidas.

2. **Desafios Principais**:
   - Entender quando usar pilhas e filas de maneira adequada com base no contexto da questão.
   - Implementar as operações corretamente respeitando a lógica dessas estruturas.

3. **Dicas Contextualizadas**:
   - Pense em uma fila como uma linha de pessoas esperando para entrar em um cinema: você entra no final da fila e sai pela frente.
   - Para uma pilha, imagine uma pilha de pratos: você coloca um novo prato em cima da pilha e retira o prato que está no topo.

4. **Passo a Passo para a Resolução**:
   - **Entenda as operações**: Faça uma lista das operações que você prec

2025-06-07 16:11:34,209 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-07 16:11:34,210 - INFO - ------------ Questão 138/139 processada com sucesso. ------------
2025-06-07 16:11:34,211 - ERROR - Erro ao processar questão 138: name 'time' is not defined


Assunto: Algoritmos e Estruturas de Dados, Subassunto: Programação dinâmica com máscara de bits (por exemplo, encontrar caminho Hamiltoniano), Nível: Difícil
Resposta: Para abordar a questão relacionada à programação dinâmica com máscara de bits, siga os passos abaixo, que irão ajudá-lo a entender como iniciar a resolução do problema:

1. **Leitura e Identificação dos Desafios**:
   - Comece lendo atentamente o enunciado da questão, identificando quais informações foram fornecidas e quais resultados são esperados. Questões de programação dinâmica geralmente envolvem decisões otimizadas sobre o estado atual e as transições entre esses estados.

2. **Entendimento da Abordagem com Máscara de Bits**:
   - A programação com máscara de bits é útil quando você precisa acompanhar um subconjunto de elementos. Pense nela como uma forma de representar a presença ou ausência de itens de maneira compacta. Por exemplo, se você tem 3 itens, poderia usar um número de 3 bits (como 101) para indicar que

2025-06-07 16:11:44,973 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-07 16:11:44,973 - INFO - ------------ Questão 139/139 processada com sucesso. ------------
2025-06-07 16:11:44,974 - ERROR - Erro ao processar questão 139: name 'time' is not defined
2025-06-07 16:11:44,979 - INFO - Arquivo salvo com sucesso em: src/data/respostas_questoes.csv


Assunto: Algoritmos e Estruturas de Dados, Subassunto: Estratégias simples de iteração e repetição, Nível: Fácil
Resposta: 1. **Identificação dos Desafios**:
   - Ler e entender o enunciado é crucial. Normalmente, as questões de algoritmos simples de iteração requerem que você faça um loop sobre uma ou mais coleções de dados, adotando uma abordagem sistemática para processar cada item.
   - Atenção às condições de parada e como os dados estão organizados.

2. **Dicas Contextualizadas**:
   - Pense em um exemplo do dia a dia, como contar maçãs em uma cesta. Você começaria olhando uma maçã, contando-a e, em seguida, moveria para a próxima até que todas as maçãs fossem contadas. Isso se assemelha à maneira de iterar sobre uma lista de números ou elementos.

3. **Passo a Passo**:
   - **Passo 1**: Compreende o que a questão pede. Identifique as variáveis que você precisa monitorar (por exemplo, totais, contagens).
   - **Passo 2**: Prepare-se para iterar. Para cada elemento da sua entrada,